In [1]:
# load libraries
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action="ignore", category=Warning)

In [2]:
df = pd.read_csv("Article_top_Data.csv")
df.head()

,Unnamed: 0,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,ReceiptRowNumber,Time,ArticleNumber,ArticleName,Quantity,Timestamp,Month,Day,Hour,Group
0,4,2022-01-03,1001,1,296,5,09:21:14,419727,"GL H-Milch 3,5% 1L VLOG",1.0,2022-01-03 09:21:14,1,0,9,Milch
1,6,2022-01-03,1001,1,296,7,09:21:14,999999998025,Pfandartikel (Tandem) - EW,1.0,2022-01-03 09:21:14,1,0,9,Pfandartikel
2,7,2022-01-03,1001,1,296,8,09:21:14,435066,Schlaufentragetasche ST,1.0,2022-01-03 09:21:14,1,0,9,Schlaufentragetasche ST
3,19,2022-01-03,1001,1,300,1,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST
4,20,2022-01-03,1001,1,300,2,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST


In [3]:
df = df.groupby(['Date', 'StoreNumber', 'CashRegisterNumber', 'ReceiptNumber', 'Group'])['Quantity'].sum().reset_index()
df.head()

,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,Group,Quantity
0,2022-01-03,1001,1,296,Milch,1.0
1,2022-01-03,1001,1,296,Pfandartikel,1.0
2,2022-01-03,1001,1,296,Schlaufentragetasche ST,1.0
3,2022-01-03,1001,1,300,BO-Laugenbreze 1ST,2.0
4,2022-01-03,1001,1,302,Avocado vorgereift ST,1.0


In [4]:
# drop articles with overwhelming quantity
df.drop(df[df.Group == "Pfandartikel"].index, inplace=True)
df.head()

,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,Group,Quantity
0,2022-01-03,1001,1,296,Milch,1.0
2,2022-01-03,1001,1,296,Schlaufentragetasche ST,1.0
3,2022-01-03,1001,1,300,BO-Laugenbreze 1ST,2.0
4,2022-01-03,1001,1,302,Avocado vorgereift ST,1.0
5,2022-01-03,1001,1,303,VL Eier BH 10ST,1.0


In [5]:
df['Duplicates'] = df.groupby(['Date', 'StoreNumber', 'CashRegisterNumber', 'ReceiptNumber'])['Date'].transform('count')
df = df[df['Duplicates'] >= 4]
df.drop(columns=['Duplicates'], inplace=True)
df.head()

,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,Group,Quantity
12,2022-01-03,1001,2,4,Avocado vorgereift ST,2.0
13,2022-01-03,1001,2,4,Broetchen,6.0
14,2022-01-03,1001,2,4,GL Sahne 30% 200g VLOG,2.0
15,2022-01-03,1001,2,4,Gurken ST,3.0
16,2022-01-03,1001,2,4,VL Eier BH 10ST,1.0


In [6]:
df['User'] = df['Date'].astype(str) + '_' + df['StoreNumber'].astype(str) + '_' + df['CashRegisterNumber'].astype(str) + '_' + df['ReceiptNumber'].astype(str)
df.drop(columns=['Date', 'StoreNumber', 'CashRegisterNumber', 'ReceiptNumber'], inplace=True)
df.head()

,Group,Quantity,User
12,Avocado vorgereift ST,2.0,2022-01-03_1001_2_4
13,Broetchen,6.0,2022-01-03_1001_2_4
14,GL Sahne 30% 200g VLOG,2.0,2022-01-03_1001_2_4
15,Gurken ST,3.0,2022-01-03_1001_2_4
16,VL Eier BH 10ST,1.0,2022-01-03_1001_2_4


In [7]:
user_total_quantity = df.groupby('User')['Quantity'].sum()
user_total_items = df.groupby('User')['Group'].count()
df['Rating'] = df.apply(lambda row: row['Quantity'] / (user_total_quantity[row['User']]) + 0.5 + 1/user_total_items[row['User']], axis=1)
df.head()

,Group,Quantity,User,Rating
12,Avocado vorgereift ST,2.0,2022-01-03_1001_2_4,0.842857
13,Broetchen,6.0,2022-01-03_1001_2_4,1.128571
14,GL Sahne 30% 200g VLOG,2.0,2022-01-03_1001_2_4,0.842857
15,Gurken ST,3.0,2022-01-03_1001_2_4,0.914286
16,VL Eier BH 10ST,1.0,2022-01-03_1001_2_4,0.771429


In [8]:
# df['Rating'] = df['Rating'].apply(lambda x: 1 if x > 1 else x)
df.Rating.max()

1.6547044632086854

In [9]:
df.rename(columns={'Group':'Item'}, inplace=True)
df.drop(columns='Quantity', inplace=True)
df = df[[df.columns[1], df.columns[0]] + list(df.columns[2:])]
df.head()

,User,Item,Rating
12,2022-01-03_1001_2_4,Avocado vorgereift ST,0.842857
13,2022-01-03_1001_2_4,Broetchen,1.128571
14,2022-01-03_1001_2_4,GL Sahne 30% 200g VLOG,0.842857
15,2022-01-03_1001_2_4,Gurken ST,0.914286
16,2022-01-03_1001_2_4,VL Eier BH 10ST,0.771429


In [10]:
df.shape

(66555, 3)

In [11]:
# 将未购买的商品评分设为0
all_items = set(df['Item'].unique())
for user in df['User'].unique():
    user_items = set(df[df['User'] == user]['Item'])
    for item in all_items - user_items:
        df = df.append({'User': user, 'Item': item, 'Rating': 0}, ignore_index=True)
df.head()

,User,Item,Rating
0,2022-01-03_1001_2_4,Avocado vorgereift ST,0.842857
1,2022-01-03_1001_2_4,Broetchen,1.128571
2,2022-01-03_1001_2_4,GL Sahne 30% 200g VLOG,0.842857
3,2022-01-03_1001_2_4,Gurken ST,0.914286
4,2022-01-03_1001_2_4,VL Eier BH 10ST,0.771429


In [12]:
# df.to_csv("rating_data2.csv")
df.to_csv("rating_data3.csv")
